In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [66]:
zoo_data = pd.read_csv("./zoo.csv")
zoo_data.head()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [67]:
zoo_data.tail()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
96,wallaby,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,1
97,wasp,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,6
98,wolf,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
99,worm,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,7
100,wren,0,1,1,0,1,0,0,0,1,1,0,0,2,1,0,0,2


In [68]:
def relu_activation(data_array):
    return np.maximum(0, data_array)

def softmax(output_array):
    logits = np.exp(output_array)
    return logits / np.sum(logits, axis = 1, keepdims = True)

In [69]:
# Learning Rate and epochs
LR = 0.01
EP = 1000

In [70]:
def back_propagation(ins, outs, weight_hidden, weight_out, bias_hidden, bias_out, activation_val, y):
    # Calculate Gradients 
    softmax_gradient = (y - outs) / outs.shape[0]
    weight_out_gradient = np.dot(activation_val.T, softmax_gradient)
    activation_val_gradient = np.dot(softmax_gradient, weight_out.T)
    activation_val_gradient[activation_val <= 0] = 0
    weight_hidden_gradient = np.dot(ins.T, activation_val_gradient)
    bias_out_gradient = np.sum(softmax_gradient, axis = 0, keepdims = True)
    bias_hidden_gradient = np.sum(activation_val_gradient, axis = 0, keepdims = True)
    # Update layers and biases weights     
    weight_out -= LR * weight_out_gradient
    weight_hidden -= LR * weight_hidden_gradient
    bias_out -= LR * bias_out_gradient 
    bias_hidden -= LR * bias_hidden_gradient

In [71]:
def multilayer_perceptrons(ins, outs, width):
    np.random.seed(42)
    weight_hidden = np.random.rand(len(ins[0]), width)
    weight_out = np.random.rand(width, 7)
    bias_hidden = np.zeros((1, width))
    bias_out = np.zeros((1, 7))
    for e in range(1, EP + 1):
        #Feed forward         
        z_delta = np.dot(ins, weight_hidden) + bias_hidden
        activation_val = relu_activation(z_delta)
        z_out = np.dot(activation_val, weight_out) + bias_out
        y = softmax(z_out)
        #cross_entropy         
        indices = np.argmax(outs, axis = 1).astype(int)
        predict_prob = y[np.arange(len(y)), indices]
        cross_entropy = -np.sum(np.log(predict_prob)) / len(np.log(predict_prob))        
    
        back_propagation(ins, outs, weight_hidden, weight_out, bias_hidden, bias_out, activation_val, y)
    
        if e == EP:
            dst = {}
            dst['WH'], dst['WO'], dst['BH'], dst['BO'] = weight_hidden, weight_out, bias_hidden, bias_out
            print("With %d epochs (EP), cross entropy: %f" % (e, cross_entropy))
    
    return dst

In [77]:
ins = zoo_data.drop(columns=['animal_name', 'class_type'])
outs = pd.get_dummies(zoo_data['class_type'].values).values

i = round(.6 * len(ins))
in_train_set = np.array(ins[:i])
out_train_set = np.array(outs[:i])
in_test_set = np.array(ins[i:])
out_test_set = np.array(outs[i:])

model_scores = multilayer_perceptrons(ins = in_train_set, outs = out_train_set, width = 16)

With 1000 epochs (EP), cross entropy: 0.091327


In [73]:
def accuracy(ins, outs, weight_hidden, weight_out, bias_hidden, bias_out):
    z_delta = np.dot(ins, weight_hidden) + bias_hidden
    activation_val = relu_activation(z_delta)
    z_out = np.dot(activation_val, weight_out) + bias_out
    y = softmax(z_out)
    correct_predictions = np.sum(np.argmax(y, 1) == np.argmax(outs, 1))
    return correct_predictions / len(outs)

In [74]:
accuracy_res = accuracy(ins = in_test_set, outs = out_test_set, weight_hidden = model_scores['WH'], weight_out = model_scores['WO'], bias_hidden = model_scores['BH'], bias_out = model_scores['BO'])

In [75]:
print("The Accuracy of this model is: ", round(accuracy_res * 100),"%")

The Accuracy of this model is:  78 %
